In [253]:
import random
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from copy import deepcopy
import data_generation as dg
import time

In [254]:
graph, random_solution, total_edge_length = dg.read_instance('data\data8.txt')
num_nodes = len(graph)

In [255]:
sols = []
values = []
iters = []
times = []

<span style="font-size: larger;">Classic local search</span>

In [256]:
def make_change_swap(graph, solution):
    new_solution = solution.copy()
    
    # Choose two distinct random indices
    index1, index2 = random.sample(range(len(solution)), 2)
    
    # Swap the positions of the selected nodes
    new_solution[index1], new_solution[index2] = new_solution[index2], new_solution[index1]

    return new_solution

In [257]:
def make_change_inverse(graph, solution):
    new_solution = solution.copy()
    
    # Choose two distinct random indices
    index1, index2 = sorted(random.sample(range(len(solution)), 2))
    
    # Reverse the subset of nodes between index1 and index2
    new_solution[index1:index2+1] = reversed(new_solution[index1:index2+1])

    return new_solution

In [258]:
def make_change_next_permutation(graph, solution):
    #based on the classic next permutation algorithm
    new_solution = deepcopy(solution)
    
    n = len(new_solution)
    
    # Find the largest index k such that a[k] < a[k+1]
    k = n - 2
    while k >= 0 and new_solution[k] >= new_solution[k + 1]:
        k -= 1

    # If no such index exists, the permutation is the last one
    if k == -1:
        return sorted(new_solution)

    # Find the largest index l greater than k such that a[k] < a[l]
    l = n - 1
    while new_solution[k] >= new_solution[l]:
        l -= 1

    # Swap a[k] and a[l]
    new_solution[k], new_solution[l] = new_solution[l], new_solution[k]

    # Reverse the sequence from a[k+1] up to and including the final element a[n-1]
    new_solution[k + 1:] = reversed(new_solution[k + 1:])

    return new_solution


In [259]:
def make_change_scramble(graph, solution):
    new_solution = deepcopy(solution)
    
    start, end = sorted(random.sample(range(len(solution)), 2))

    subset = solution[start:end+1]

    random.shuffle(subset)

    new_solution[start:end+1] = subset

    return new_solution

In [260]:
def local_search(graph, random_solution, value, num_iters, change_func):
    solution = deepcopy(random_solution)
    best_solution = deepcopy(solution)
    best_value = value
    best_i = None

    for i in range(num_iters):
        #print(solution, value, i)
        new_solution = change_func(graph, solution)
        new_value = dg.calculate_total_edge_length(graph, new_solution)

        if new_value < value:
            value = new_value
            solution = deepcopy(new_solution)

            if new_value < best_value:
                best_i = i
                best_value = new_value
                best_solution = deepcopy(new_solution)

    return best_solution, best_value, best_i

In [261]:
start = time.time()
solution, value, iter = local_search(graph, random_solution, total_edge_length, 500, change_func=make_change_swap)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[5, 1, 6, 7, 0, 4, 3, 2] 28 87


In [262]:
start = time.time()
solution, value, iter = local_search(graph, random_solution, total_edge_length, 500, change_func=make_change_inverse)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[5, 1, 3, 0, 4, 6, 7, 2] 29 23


In [263]:
start = time.time()
solution, value, iter = local_search(graph, random_solution, total_edge_length, 500, change_func=make_change_scramble)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[5, 1, 7, 6, 0, 4, 3, 2] 28 57


In [264]:
#generating comparison table
from IPython.display import display
import pandas as pd
dim = num_nodes

results = []

best = float('inf')
best_i = -1

average = [0,0,0]

number_of_combinations = 0

solution_generation = [make_change_swap, make_change_inverse, make_change_scramble]
for i, sg in enumerate(solution_generation):
    method = sg.__name__
    results.append({'Dim': dim, 'Method': method, 'Value': values[i], 'Time': times[i], 'Best_Iter': iters[i]})

    if values[i] <= best:
        if times[i] <= times[best_i]:
            best = values[i]
            best_i = i

    average[0] += values[i]
    average[1] += times[i] 
    average[2] += iters[i]

    number_of_combinations += 1

df = pd.DataFrame(results)
#display(df)
display(df.drop('Best_Iter', axis=1)) 

,Dim,Method,Value,Time
0,8,make_change_swap,28,0.01
1,8,make_change_inverse,29,0.01
2,8,make_change_scramble,28,0.02


In [265]:
print('best:', solution_generation[best_i].__name__, values[best_i], times[best_i], iters[best_i])
df_best = pd.DataFrame({'Dim': dim, 'Method':  solution_generation[best_i].__name__, 'Value': values[best_i], 'Time': times[best_i], 'Best_Iter': iters[best_i]}, index=[0])
display(df_best)

best: make_change_swap 28 0.01 87


,Dim,Method,Value,Time,Best_Iter
0,8,make_change_swap,28,0.01,87


In [266]:
df = pd.read_csv('comparison_tables/bests.csv')
row_to_update = df[df['Dim'] == num_nodes]

# Check if the row exists
if not row_to_update.empty:
    # Update specific columns in the located row
    df.loc[row_to_update.index, 'local_search'] = values[best_i]
    df.loc[row_to_update.index, 'local_search_time'] = times[best_i]

    # Save the updated DataFrame back to the CSV file
    df.to_csv('comparison_tables/bests.csv', index=False)
else:
    print(f"Row with Dim={num_nodes} not found.")

In [267]:
average = [average[0] / number_of_combinations, average[1] / number_of_combinations, average[2] / number_of_combinations]
average = [round(num, 2) for num in average]
print('average:', average)
df_avg = pd.DataFrame({'Dim': dim, 'Value': average[0], 'Time': average[1], 'Best_Iter': average[2]}, index=[0])
display(df_avg)

average: [28.33, 0.01, 55.67]


,Dim,Value,Time,Best_Iter
0,8,28.33,0.01,55.67


<span style="font-size: larger;">Local search with next permutation help</span>

In [268]:
sols = []
values = []
iters = []
times = []

def local_search_with_permutation(graph, random_solution, value, num_iters, change_func):
    solution = deepcopy(random_solution)
    best_solution = deepcopy(solution)
    best_value = value
    best_i = None

    for i in range(num_iters):
        #print(solution, value, i)
        new_solution = change_func(graph, solution)
        new_value = dg.calculate_total_edge_length(graph, new_solution)

        if new_value < value:
            value = new_value
            solution = deepcopy(new_solution)

            if new_value < best_value:
                best_i = i
                best_value = new_value
                best_solution = deepcopy(new_solution)

        else:
            perm_counter = 0
            perm_limit = num_iters / (i+1)
            while new_value >= value and perm_counter < perm_limit:    
                new_solution = make_change_next_permutation(graph, new_solution)
                new_value = dg.calculate_total_edge_length(graph, new_solution)
                perm_counter += 1

            if new_value < value:
                value = new_value
                solution = deepcopy(new_solution)

                if new_value < best_value:
                    best_i = i
                    best_value = new_value
                    best_solution = deepcopy(new_solution)
              

    return best_solution, best_value, best_i

In [269]:
start = time.time()
solution, value, iter = local_search_with_permutation(graph, random_solution, total_edge_length, 500, change_func=make_change_inverse)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[5, 1, 3, 2, 4, 7, 0, 6] 26 102


In [270]:
start = time.time()
solution, value, iter = local_search_with_permutation(graph, random_solution, total_edge_length, 500, change_func=make_change_swap)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[6, 0, 7, 4, 2, 3, 1, 5] 26 58


In [271]:
start = time.time()
solution, value, iter = local_search_with_permutation(graph, random_solution, total_edge_length, 500, change_func=make_change_scramble)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[2, 3, 4, 0, 6, 7, 1, 5] 28 11


In [272]:
#generating comparison table
from IPython.display import display
import pandas as pd
dim = num_nodes

results = []

best = float('inf')
best_i = -1

average = [0,0,0]

number_of_combinations = 0

solution_generation = [make_change_swap, make_change_inverse, make_change_scramble]
for i, sg in enumerate(solution_generation):
    method = sg.__name__ + '_with_permutation'
    results.append({'Dim': dim, 'Method': method, 'Value': values[i], 'Time': times[i], 'Best_Iter': iters[i]})

    if values[i] <= best:
        if times[i] <= times[best_i]:
            best = values[i]
            best_i = i

    average[0] += values[i]
    average[1] += times[i] 
    average[2] += iters[i]

    number_of_combinations += 1

df = pd.DataFrame(results)
#display(df)
display(df.drop('Best_Iter', axis=1)) 

,Dim,Method,Value,Time
0,8,make_change_swap_with_permutation,26,0.07
1,8,make_change_inverse_with_permutation,26,0.07
2,8,make_change_scramble_with_permutation,28,0.08


In [273]:
print('best:', solution_generation[best_i].__name__, values[best_i], times[best_i], iters[best_i])
df_best = pd.DataFrame({'Dim': dim, 'Method':  solution_generation[best_i].__name__ + '_with_permutation', 'Value': values[best_i], 'Time': times[best_i], 'Best_Iter': iters[best_i]}, index=[0])
display(df_best)

best: make_change_inverse 26 0.07 58


,Dim,Method,Value,Time,Best_Iter
0,8,make_change_inverse_with_permutation,26,0.07,58


In [274]:
df = pd.read_csv('comparison_tables/bests.csv')
row_to_update = df[df['Dim'] == num_nodes]

# Check if the row exists
if not row_to_update.empty:
    # Update specific columns in the located row
    df.loc[row_to_update.index, 'local_search_perm'] = values[best_i]
    df.loc[row_to_update.index, 'local_search_perm_time'] = times[best_i]

    # Save the updated DataFrame back to the CSV file
    df.to_csv('comparison_tables/bests.csv', index=False)
else:
    print(f"Row with Dim={num_nodes} not found.")

In [275]:
average = [average[0] / number_of_combinations, average[1] / number_of_combinations, average[2] / number_of_combinations]
average = [round(num, 2) for num in average]
print('average:', average)
df_avg = pd.DataFrame({'Dim': dim, 'Value': average[0], 'Time': average[1], 'Best_Iter': average[2]}, index=[0])
display(df_avg)

average: [26.67, 0.07, 57.0]


,Dim,Value,Time,Best_Iter
0,8,26.67,0.07,57.0


<span style="font-size: larger;">Simulated annealing</span>

In [276]:
sols = []
values = []
iters = []
times = []

def simulated_annealing(graph, random_solution, value, num_iters, change_func):
    solution = deepcopy(random_solution)
    best_solution = deepcopy(solution)
    best_value = value
    best_i = None

    for i in range(1, num_iters + 1):
        #print(solution, value)
        new_solution = change_func(graph, solution)
        new_value = dg.calculate_total_edge_length(graph, new_solution)

        if new_value < value:
            value = new_value
            solution = deepcopy(new_solution)

            if new_value < best_value:
                best_i = i
                best_value = new_value
                best_solution = deepcopy(new_solution)

        #elif random.random() < 1 / (i**0.5):
        elif random.random() < 1 / i:
            #print('divs')
            value = new_value
            solution = deepcopy(new_solution)

    return best_solution, best_value, best_i

In [277]:
start = time.time()
solution, value, iter = simulated_annealing(graph, random_solution, total_edge_length, 500, change_func=make_change_swap)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[2, 3, 4, 0, 7, 6, 1, 5] 28 181


In [278]:
start = time.time()
solution, value, iter = simulated_annealing(graph, random_solution, total_edge_length, 500, change_func=make_change_inverse)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[5, 1, 3, 2, 4, 7, 0, 6] 26 188


In [279]:
start = time.time()
solution, value, iter = simulated_annealing(graph, random_solution, total_edge_length, 500, change_func=make_change_scramble)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[5, 1, 3, 2, 7, 4, 0, 6] 26 101


In [280]:
#generating comparison table
from IPython.display import display
import pandas as pd
dim = num_nodes

results = []

best = float('inf')
best_i = -1

average = [0,0,0]

number_of_combinations = 0

solution_generation = [make_change_swap, make_change_inverse, make_change_scramble]
for i, sg in enumerate(solution_generation):
    method = sg.__name__
    results.append({'Dim': dim, 'Method': 'simulated_annealing' + ', '  + method, 'Value': values[i], 'Time': times[i], 'Best_Iter': iters[i]})

    if values[i] <= best:
        if times[i] <= times[best_i]:
            best = values[i]
            best_i = i

    average[0] += values[i]
    average[1] += times[i] 
    average[2] += iters[i]

    number_of_combinations += 1

df = pd.DataFrame(results)
#display(df)
display(df.drop('Best_Iter', axis=1)) 

,Dim,Method,Value,Time
0,8,"simulated_annealing, make_change_swap",28,0.01
1,8,"simulated_annealing, make_change_inverse",26,0.01
2,8,"simulated_annealing, make_change_scramble",26,0.02


In [281]:
print('best:', solution_generation[best_i].__name__, values[best_i], times[best_i], iters[best_i])
df_best = pd.DataFrame({'Dim': dim, 'Method':  'simulated_annealing' + ', '  + solution_generation[best_i].__name__, 'Value': values[best_i], 'Time': times[best_i], 'Best_Iter': iters[best_i]}, index=[0])
display(df_best)

best: make_change_inverse 26 0.01 188


,Dim,Method,Value,Time,Best_Iter
0,8,"simulated_annealing, make_change_inverse",26,0.01,188


In [282]:
df = pd.read_csv('comparison_tables/bests.csv')
row_to_update = df[df['Dim'] == num_nodes]

# Check if the row exists
if not row_to_update.empty:
    # Update specific columns in the located row
    df.loc[row_to_update.index, 'sim_annealing'] = values[best_i]
    df.loc[row_to_update.index, 'sim_annealing_time'] = times[best_i]

    # Save the updated DataFrame back to the CSV file
    df.to_csv('comparison_tables/bests.csv', index=False)
else:
    print(f"Row with Dim={num_nodes} not found.")

In [283]:
average = [average[0] / number_of_combinations, average[1] / number_of_combinations, average[2] / number_of_combinations]
average = [round(num, 2) for num in average]
print('average:', average)
df_avg = pd.DataFrame({'Dim': dim, 'Value': average[0], 'Time': average[1], 'Best_Iter': average[2]}, index=[0])
display(df_avg)

average: [26.67, 0.01, 156.67]


,Dim,Value,Time,Best_Iter
0,8,26.67,0.01,156.67


<span style="font-size: larger;">Variable neigborhood search</span>

In [284]:
def shaking_swap(graph, solution, k):
    new_solution = deepcopy(solution)
    selected = random.sample(range(len(solution)), 2*k)
    for i in range(0,len(selected), 2):
        index1, index2 = selected[i], selected[i+1]
        new_solution[index1], new_solution[index2] = new_solution[index2], new_solution[index1]
    return new_solution    
    

In [285]:
def shaking_inverse(graph, solution, k):
    new_solution = deepcopy(solution)
    index1 = random.randint(0, len(solution))
    if(index1 + k > len(solution)):
        index1 = len(solution) - k
    index2 = index1 + k                       
    new_solution[index1:index2+1] = reversed(new_solution[index1:index2+1])  

    return new_solution

In [286]:
def vns(graph, random_solution, value, num_iters, change_func, shaking_func, local_search_func, k_min, k_max, move_prob):
    solution = deepcopy(random_solution)
    best_i = None
    for i in range(num_iters):
        for k in range(k_min, k_max):
            #print('vns: ', solution, value, i)
            new_solution = shaking_func(graph, solution, k) #diversification  

            new_value = dg.calculate_total_edge_length(graph, new_solution)

            #print('post shaking: ', new_solution, new_value, i)
            new_solution, new_value, _= local_search_func(graph, new_solution, total_edge_length, 10, change_func)

            if new_value < value or (new_value == value and random.random() < move_prob):
                if(new_value < value):
                    best_i = i

                value = new_value

                solution = deepcopy(new_solution)
                
    return solution, value, best_i

In [287]:
start = time.time()
solution, value, iter = vns(graph, random_solution, total_edge_length, 500, change_func=make_change_inverse, shaking_func=shaking_swap, local_search_func = local_search, k_min=1, k_max=3, move_prob=0.4)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[6, 0, 4, 7, 2, 3, 1, 5] 26 29


In [288]:
start = time.time()
solution, value, iter = vns(graph, random_solution, total_edge_length, 500, change_func=make_change_inverse, shaking_func=shaking_inverse, local_search_func = local_search, k_min=3, k_max=6, move_prob=0.4)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[5, 1, 3, 2, 7, 4, 0, 6] 26 10


In [289]:
start = time.time()
solution, value, iter = vns(graph, random_solution, total_edge_length, 500, change_func=make_change_swap, shaking_func=shaking_swap, local_search_func = local_search, k_min=1, k_max=3, move_prob=0.4)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[6, 0, 7, 4, 2, 3, 1, 5] 26 19


In [290]:
start = time.time()
solution, value, iter = vns(graph, random_solution, total_edge_length, 500, change_func=make_change_swap, shaking_func=shaking_inverse, local_search_func = local_search, k_min=3, k_max=6, move_prob=0.4)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[6, 0, 4, 7, 2, 3, 1, 5] 26 49


In [291]:
start = time.time()
solution, value, iter = vns(graph, random_solution, total_edge_length, 500, change_func=make_change_scramble, shaking_func=shaking_swap, local_search_func = local_search, k_min=1, k_max=3, move_prob=0.4)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[5, 1, 3, 2, 7, 4, 0, 6] 26 16


In [292]:
start = time.time()
solution, value, iter = vns(graph, random_solution, total_edge_length, 500, change_func=make_change_scramble, shaking_func=shaking_inverse, local_search_func = local_search, k_min=3, k_max=6, move_prob=0.4)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[6, 0, 7, 4, 2, 3, 1, 5] 26 32


In [293]:
start = time.time()
solution, value, iter = vns(graph, random_solution, total_edge_length, 500, change_func=make_change_swap, shaking_func=shaking_swap, local_search_func = local_search_with_permutation, k_min=1, k_max=3, move_prob=0.4)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[6, 0, 4, 7, 2, 3, 1, 5] 26 1


In [294]:
start = time.time()
solution, value, iter = vns(graph, random_solution, total_edge_length, 500, change_func=make_change_swap, shaking_func=shaking_inverse, local_search_func = local_search_with_permutation, k_min=3, k_max=6, move_prob=0.4)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[5, 1, 3, 2, 7, 4, 0, 6] 26 2


In [295]:
start = time.time()
solution, value, iter = vns(graph, random_solution, total_edge_length, 500, change_func=make_change_inverse, shaking_func=shaking_swap, local_search_func = local_search_with_permutation, k_min=1, k_max=3, move_prob=0.4)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[5, 1, 3, 2, 4, 7, 0, 6] 26 4


In [296]:
start = time.time()
solution, value, iter = vns(graph, random_solution, total_edge_length, 500, change_func=make_change_inverse, shaking_func=shaking_inverse, local_search_func = local_search_with_permutation, k_min=3, k_max=6, move_prob=0.4)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[6, 0, 7, 4, 2, 3, 1, 5] 26 12


In [297]:
start = time.time()
solution, value, iter = vns(graph, random_solution, total_edge_length, 500, change_func=make_change_scramble, shaking_func=shaking_swap, local_search_func = local_search_with_permutation, k_min=1, k_max=3, move_prob=0.4)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[5, 1, 3, 2, 4, 7, 0, 6] 26 4


In [298]:
start = time.time()
solution, value, iter = vns(graph, random_solution, total_edge_length, 500, change_func=make_change_scramble, shaking_func=shaking_inverse, local_search_func = local_search_with_permutation, k_min=3, k_max=6, move_prob=0.4)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[6, 0, 7, 4, 2, 3, 1, 5] 26 2


In [299]:
n = len(graph)

oc = False

shakings = [shaking_swap, shaking_inverse]
changes_funcs = [make_change_swap, make_change_inverse, make_change_scramble]
search_funcs = [local_search, local_search_with_permutation]

number_of_combinations = 0

sols = []
values =[]
times = []

methods = []

results = []

best = float('inf')
best_i = -1

average = [0,0]

k_min = int(0.1 * num_nodes)
k_max = int(0.3 * num_nodes)
for shake in shakings:
    for change in changes_funcs:
        for search in search_funcs:
            if shake.__name__ == 'shaking_inverse':
                k_min = int(0.3 * num_nodes)
                k_max = int(0.6 * num_nodes)

            start = time.time()    
            sol, value, iter = vns(graph, 
                                   random_solution, 
                                   total_edge_length, 
                                   500, 
                                   change_func=change, 
                                   shaking_func=shake, 
                                   local_search_func = search, 
                                   k_min=k_min,
                                   k_max=k_max, 
                                   move_prob=0.4)
            end = time.time()
            duration = float("{:.2f}".format(end - start))
            
            print(shake.__name__, change.__name__, search.__name__, ' : ', sol, value, iter, '\n')
            
            k_min = int(0.1 * num_nodes)
            k_max = int(0.3 * num_nodes)

            sols.append(sol)
            values.append(value)
            iters.append(iter)
            times.append(duration)

            methods.append(str(shake.__name__ + ', ' + change.__name__ + ', ' + search.__name__))

            results.append({'Dim': num_nodes, 'Method': methods[number_of_combinations], 'Value': abs(value), 'Time': duration})

            if value <= best:
                if duration <= times[best_i]:    
                    best = value
                    best_i = number_of_combinations

            average[0] += value
            average[1] += duration

            number_of_combinations += 1    


shaking_swap make_change_swap local_search  :  [6, 0, 4, 7, 2, 3, 1, 5] 26 4 

shaking_swap make_change_swap local_search_with_permutation  :  [6, 0, 7, 4, 2, 3, 1, 5] 26 5 

shaking_swap make_change_inverse local_search  :  [6, 0, 7, 4, 2, 3, 1, 5] 26 48 

shaking_swap make_change_inverse local_search_with_permutation  :  [5, 1, 3, 2, 4, 7, 0, 6] 26 1 

shaking_swap make_change_scramble local_search  :  [5, 1, 3, 2, 4, 7, 0, 6] 26 12 

shaking_swap make_change_scramble local_search_with_permutation  :  [6, 0, 4, 7, 2, 3, 1, 5] 26 4 

shaking_inverse make_change_swap local_search  :  [6, 0, 7, 4, 2, 3, 1, 5] 26 16 

shaking_inverse make_change_swap local_search_with_permutation  :  [6, 0, 7, 4, 2, 3, 1, 5] 26 21 

shaking_inverse make_change_inverse local_search  :  [5, 1, 3, 2, 4, 7, 0, 6] 26 32 

shaking_inverse make_change_inverse local_search_with_permutation  :  [5, 1, 3, 2, 7, 4, 0, 6] 26 23 

shaking_inverse make_change_scramble local_search  :  [5, 1, 3, 2, 7, 4, 0, 6] 26 2 

s

In [300]:
df = pd.DataFrame(results)
display(df)
df.to_csv('comparison_tables/vns.csv', mode='a', header=not pd.io.common.file_exists('comparison_tables/vns.csv'), index=False)

,Dim,Method,Value,Time
0,8,"shaking_swap, make_change_swap, local_search",26,0.33
1,8,"shaking_swap, make_change_swap, local_search_w...",26,0.82
2,8,"shaking_swap, make_change_inverse, local_search",26,0.32
3,8,"shaking_swap, make_change_inverse, local_searc...",26,0.81
4,8,"shaking_swap, make_change_scramble, local_search",26,0.44
5,8,"shaking_swap, make_change_scramble, local_sear...",26,0.97
6,8,"shaking_inverse, make_change_swap, local_search",26,0.28
7,8,"shaking_inverse, make_change_swap, local_searc...",26,0.79
8,8,"shaking_inverse, make_change_inverse, local_se...",26,0.31
9,8,"shaking_inverse, make_change_inverse, local_se...",26,0.84


In [301]:
print('best:', methods[best_i], values[best_i], times[best_i])
df_best = pd.DataFrame({'Dim': num_nodes, 'Method':  methods[best_i], 'Value': values[best_i], 'Time': times[best_i]}, index=[0])
display(df_best)

best: shaking_inverse, make_change_swap, local_search 26 0.28


,Dim,Method,Value,Time
0,8,"shaking_inverse, make_change_swap, local_search",26,0.28


In [302]:
df = pd.read_csv('comparison_tables/bests.csv')
row_to_update = df[df['Dim'] == num_nodes]

# Check if the row exists
if not row_to_update.empty:
    # Update specific columns in the located row
    df.loc[row_to_update.index, 'vns'] = values[best_i]
    df.loc[row_to_update.index, 'vns_time'] = times[best_i]

    # Save the updated DataFrame back to the CSV file
    df.to_csv('comparison_tables/bests.csv', index=False)
else:
    print(f"Row with Dim={num_nodes} not found.")

In [303]:
average = [average[0] / number_of_combinations, average[1] / number_of_combinations]
average = [round(num, 2) for num in average]
print('average:', average)
df_avg = pd.DataFrame({'Dim': num_nodes, 'Value': average[0], 'Time': average[1]}, index=[0])
display(df_avg)

average: [26.0, 0.62]


,Dim,Value,Time
0,8,26.0,0.62
